<h1>410821322 Steven Jonathan</h1>

<h2>Setting up the data</h2>

<h3>Removing excessive lines at the end of horse-colic.data</h3>

In [1]:
line = open('horse-colic.data', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
# line = ' '.join(line.split())
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
# print(temp)
line = open('horse-colic.data', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

<h3>Removing excessive lines at the end of horse-colic.test</h3>

In [2]:
line = open('horse-colic.test', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
# line = ' '.join(line.split())
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
# print(temp)
line = open('horse-colic.test', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#Setting up data
headernames = ["Surgery?","Age","HospitalNo","RectalTemp","Pulse","RespiratoryRate","ExtremitiesTemp","PeripheralPulse","MucousMembranes","CapillaryRefillTime","PainLevel","Peristalsis","AbdominalDistension","NasogastricTube","NasogastricReflux","NasogastricRefluxPH","Feces","Abdomen","PackedCellVolume","TotalProtein","AbdominocentesisAppearance","AbdominocentesisTotalProtein","Outcome","SurgicalLesion","LesionTypeOne","LesionTypeTwo","LesionTypeThree","CPData"]

Needed for predictor : '''Surgery, Age, Rectal Temp, AbdominocentesisTotalProtein''' , result in Outcome

<h3>Get the data for X,Y train and also replace NaN with high number</h3>

In [4]:
horsedataoriginal = pd.read_csv("horse-colic.data",sep=" ",index_col=False,names=headernames).replace('?', 0) #change the data file path, also replace ? with NaN for easier process
horsedataoriginal = horsedataoriginal.iloc[:,:23] #We don't use 23 further, remove the factors

horsedataoriginal = horsedataoriginal.drop(columns=["HospitalNo"]) #Remove unused factors
horsedataoriginal["Outcome"] = horsedataoriginal["Outcome"].replace("3","2")
horsedataoriginal

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricTube,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome
0,2,1,38.50,66,28,3,3,0,2,5,...,0,0,0,3,5,45.00,8.40,0,0,2
1,1,1,39.2,88,20,0,0,4,1,3,...,0,0,0,4,2,50,85,2,2,2
2,2,1,38.30,40,24,1,1,3,1,3,...,0,0,0,1,1,33.00,6.70,0,0,1
3,1,9,39.10,164,84,4,1,6,2,2,...,1,2,5.00,3,0,48.00,7.20,3,5.30,2
4,2,1,37.30,104,35,0,0,6,2,0,...,0,0,0,0,0,74.00,7.40,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,0,120,70,4,0,4,2,2,...,0,0,0,0,5,55,65,0,0,2
296,2,1,37.20,72,24,3,2,4,2,4,...,3,1,0,4,4,44.00,0,3,3.30,2
297,1,1,37.50,72,30,4,3,4,1,4,...,2,1,0,3,5,60.00,6.80,0,0,2
298,1,1,36.50,100,24,3,3,3,1,3,...,3,1,0,4,4,50.00,6.00,3,3.40,1


In [5]:
horsedata = horsedataoriginal
x_train = horsedata.loc[:,horsedata.columns!="Outcome"].astype(float)
y_train = horsedata["Outcome"].astype(float)

<h3>Get the data for X,Y test and also replace NaN with high number</h3>

In [6]:
horsedatatest = pd.read_csv("horse-colic.test",sep=" ",index_col=False,names=headernames).replace('?', 0)#change the data file path, also replace ? with NaN for easier process
horsedatatest = horsedatatest.iloc[:,:23]
horsedatatest = horsedatatest.drop(columns=["HospitalNo"]) #Remove unused factors
horsedatatest["Outcome"] = horsedatatest["Outcome"].replace("3","2")
horsedatatest

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricTube,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome
0,2,1,38.50,54,20,0,1,2,2,3,...,2,2,5.90,0,2,42.00,6.30,0,0,1
1,2,1,37.60,48,36,0,0,1,1,0,...,0,0,0,0,0,44.00,6.30,1,5.00,1
2,1,1,37.7,44,28,0,4,3,2,5,...,1,1,0,3,5,45,70,3,2,1
3,1,1,37,56,24,3,1,4,2,4,...,1,1,0,0,0,35,61,3,2,2
4,2,1,38.00,42,12,3,0,3,1,1,...,0,0,0,0,2,37.00,5.80,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2,1,38.60,60,30,1,1,3,1,4,...,1,1,0,0,0,40.00,6.00,1,0,1
64,2,1,37.80,42,40,1,1,1,1,1,...,0,0,0,3,3,36.00,6.20,0,0,1
65,1,1,38,60,12,1,1,2,1,2,...,1,1,0,1,4,44,65,3,2,2
66,2,1,38.00,42,12,3,0,3,1,1,...,0,0,0,0,1,37.00,5.80,0,0,1


In [7]:
horsedata = horsedatatest
x_test = horsedata.loc[:,horsedata.columns!="Outcome"].astype(float)
y_test = horsedata["Outcome"].astype(float)

<h2>Creating the KNeighborsClassifier</h2>

First, we try to see which K nearest neighbor is the best for this prediction 

In [8]:
highest = 0
highestindex = 1
for i in range(1,200):
    Kclassifier = KNeighborsClassifier(n_neighbors=i,metric='euclidean')
    Kclassifier.fit(x_train,y_train)
    y_pred = Kclassifier.predict(x_test)
    if(round(accuracy_score(y_test, y_pred)*100,2)>highest):
        highest = round(accuracy_score(y_test, y_pred)*100,2)
        highestindex=i
    print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%","in i = ",i)
    
print(f"Highest Accuracy at: {highest}% least neighbors at : {highestindex}")

Accuracy:  67.65 % in i =  1
Accuracy:  72.06 % in i =  2
Accuracy:  72.06 % in i =  3
Accuracy:  75.0 % in i =  4
Accuracy:  70.59 % in i =  5
Accuracy:  73.53 % in i =  6
Accuracy:  73.53 % in i =  7
Accuracy:  75.0 % in i =  8
Accuracy:  73.53 % in i =  9
Accuracy:  73.53 % in i =  10
Accuracy:  76.47 % in i =  11
Accuracy:  69.12 % in i =  12
Accuracy:  73.53 % in i =  13
Accuracy:  69.12 % in i =  14
Accuracy:  73.53 % in i =  15
Accuracy:  70.59 % in i =  16
Accuracy:  77.94 % in i =  17
Accuracy:  73.53 % in i =  18
Accuracy:  75.0 % in i =  19
Accuracy:  73.53 % in i =  20
Accuracy:  75.0 % in i =  21
Accuracy:  73.53 % in i =  22
Accuracy:  75.0 % in i =  23
Accuracy:  73.53 % in i =  24
Accuracy:  73.53 % in i =  25
Accuracy:  75.0 % in i =  26
Accuracy:  75.0 % in i =  27
Accuracy:  75.0 % in i =  28
Accuracy:  75.0 % in i =  29
Accuracy:  75.0 % in i =  30
Accuracy:  76.47 % in i =  31
Accuracy:  75.0 % in i =  32
Accuracy:  75.0 % in i =  33
Accuracy:  75.0 % in i =  34
Ac

In [9]:
Kclassifier = KNeighborsClassifier(n_neighbors=17,metric='euclidean')
Kclassifier.fit(x_train,y_train)
y_pred = Kclassifier.predict(x_test)
print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%")

Accuracy:  77.94 %


Here we can see that as the neighbor goes bigger, the accuracy will be less, therefore we can take the highest accuracy in which K-Neighbors, which is 77.94% when we set the n_neighbors = 17

<h2>Creating the KNNImputer</h2>


In [11]:
Kclassifier = KNNImputer(n_neighbors=2)
y_pred = Kclassifier.fit_transform(x_train,y_train)
y_pred
# y_pred = Kclassifier.predict(x_test)
# print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%")+

array([[ 2. ,  1. , 38.5, ...,  8.4,  0. ,  0. ],
       [ 1. ,  1. , 39.2, ..., 85. ,  2. ,  2. ],
       [ 2. ,  1. , 38.3, ...,  6.7,  0. ,  0. ],
       ...,
       [ 1. ,  1. , 37.5, ...,  6.8,  0. ,  0. ],
       [ 1. ,  1. , 36.5, ...,  6. ,  3. ,  3.4],
       [ 1. ,  1. , 37.2, ..., 62. ,  1. ,  1. ]])